<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/03_Okt_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pointbiserialr

# Load the "Adult" dataset from OpenML
adult = fetch_openml(name='adult', version=2, as_frame=True)
adult_data = adult.frame

# Data Preparation: Encode binary variables and filter relevant columns
adult_df = adult_data[['age', 'education-num', 'hours-per-week', 'capital-gain', 'capital-loss', 'sex', 'class']].copy()


In [2]:
adult_df.head()

,age,education-num,hours-per-week,capital-gain,capital-loss,sex,class
0,25,7,40,0,0,Male,<=50K
1,38,9,50,0,0,Male,<=50K
2,28,12,40,0,0,Male,>50K
3,44,10,40,7688,0,Male,>50K
4,18,10,30,0,0,Female,<=50K


In [4]:
df_2x2 = pd.crosstab(adult_df['sex'], adult_df['class'])
df_2x2

class,<=50K,>50K
sex,,
Female,14423,1769
Male,22732,9918


In [5]:
a = df_2x2.iloc[0,0]
b = df_2x2.iloc[0,1]
c = df_2x2.iloc[1,0]
d = df_2x2.iloc[1,1]

In [10]:
numerator = (a*d - b*c)
denominator = np.sqrt(np.longdouble(a+b) * np.longdouble(c+d) * np.longdouble(a+c) * np.longdouble(b+d))
phi_coefficient = numerator / denominator
phi_coefficient

0.2146280345639280344